In [2]:
import ae_forecasting_opendata.model

ModuleNotFoundError: No module named 'ae_forecasting_opendata'

In [4]:


from typing import Optional, Callable, Any

import numpyro
import pretty_errors
import numpyro.distributions as dist
from numpyro.infer import MCMC, NUTS, Predictive

import jax.numpy as jnp
from jax import Array, random

import plotly.express as px
import plotly.graph_objects as go
import model
from helpers import get_admissions_data

PROP_TRAIN = 0.75

In [5]:

df_admissions = get_admissions_data().loc[lambda df: df["org_code"] == "R0A"]

# Train/test split
df_admissions.sort_values(by=["org_code", "source_date"], inplace=True)
timestamps = jnp.arange(len(df_admissions.index)) #Create linspaced timestamps
admissions = jnp.array(df_admissions["ae_admissions_total"].values) #convert these to jnp array
timestamps_train, timestamps_test, admissions_train, admissions_test = (
    model.TimeSeriesModeller.train_test_split(timestamps, admissions)
)

# Fitting
modeller = model.TimeSeriesModeller(model.admissions_model, predictor_name="admissions")
modeller.fit(timestamps_train, admissions_train)
modeller.print_summary()

# Predictions
admissions_pred = modeller.predict(timestamps)["admissions"]
quantiles = [0.025, 0.05, 0.1, 0.25, 0.5, 0.75, 0.9, 0.95, 0.975]
admissions_pred_quantiles = jnp.quantile(
    admissions_pred, jnp.array(quantiles), axis=0
)

df_admissions = df_admissions.assign(
    **{
        f"ae_admissions_predicted_{q}": x
        for q, x in zip(quantiles, admissions_pred_quantiles)
    }
)

model.plot_model_results(
    x=df_admissions["source_date"].values,
    y_obs=df_admissions["ae_admissions_total"],
    y_pred=df_admissions["ae_admissions_predicted_0.5"],
    y_lower=df_admissions["ae_admissions_predicted_0.025"],
    y_upper=df_admissions["ae_admissions_predicted_0.975"],
    pred_length=timestamps_test.size,
)

sample: 100%|██████████| 1500/1500 [00:01<00:00, 975.64it/s, 15 steps of size 4.16e-01. acc. prob=0.89] 



                 mean       std    median      5.0%     95.0%     n_eff     r_hat
   gradient    259.95      4.85    260.09    251.44    267.09    426.21      1.00
  intercept   4186.49     46.46   4186.95   4110.15   4261.12    423.94      1.00
      noise    102.20      1.77    102.27     99.17    104.95    651.20      1.00

Number of divergences: 0
